This notebook shows how to integrate functions numerically using the package QuadGK.

If you have not installed it yet, first add the package QuadGK:

In [1]:
using Pkg
Pkg.add("QuadGK")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


Integrating a function is now relatively straightforward. As an example, I use
$$\int_0^1 x^2 \,dx.$$

In [3]:
using QuadGK

#define function f
f(x)=x^2

#integrate f from 0 to 1
quadgk(f,0,1)

(0.3333333333333333, 5.551115123125783e-17)

As you see, the integration command "quadgk" returns two values. The first is the actual value of the integral that we wanted to calculate $\int_0^1x^2\,dx=1/3$. The second is an estimate on the absolute error, i.e. it tells you by how much julia might have gotten it wrong. For usual functions this error is small (as here) and you can ignore it. If the second number is big, then you might not want to trust the numerical result but this will rarely happen.

Two more remarks: First, if you do not want to see the error estimate, you simply add "[1]" behind the quadgk command like so:

In [4]:
quadgk(f,0,1)[1]

0.3333333333333333

Second, you do not have to define the function over which you want to integrate in advance but you can use the quick way to define a so called anonymous function right in place:

In [5]:
quadgk(x->x^2,0,1)

(0.3333333333333333, 5.551115123125783e-17)

# Advanced: Functions with discontinuities (or other known irregularities)

If you know that your function has a discontinuity at certain points, you can tell quadgk about it and this will help quadgk. The way you tell quadgk about it is that you add points of discontinuity between the boundaries of integration in the quadgk command. As an example, I calculate the integral $\int_1^{-1} g(x)\,dx$ where 
$$g(x)=\begin{cases}1/\sqrt{x}&\text{ if }x>0\\0 & \text{ if }x=0\\-1/\sqrt{-x}&\text{ else}\end{cases}$$
which has a discontinuity at zero. (Note that this is an option. In the most usual cases quadgk will do a fine job even if your function has a discontinuity and you do not tell quadgk about it.)

In [6]:
function g(x)
    if x>0.0
        return 1.0/(sqrt(x))
    elseif x==0
        return 0.0
    else
        return -1.0/(sqrt(-x))
    end
end

quadgk(g,-1.0,0.0,1.0)

(-1.8554417678952652e-16, 4.011358198067816e-25)

Given the symmetry of the function, the true result is, of course, zero but as you see we get pretty close to that (recall that "e-16" means "multiply by $10^{-16}$"). 

# Advanced: Double integrals

There are julia packages with algorithms specialized in multidimensional integrals but a simple way of evaluating thos is to apply quadgk twice. As an example, let us calculate
$$\int_0^1\int_2^3 x/y \,dy\,dx.$$
We will do so by defining a function which evaluates the inner integral for a given $x$, i.e. 
$$h(x)=\int_2^3 x/y \,dy.$$
The original double integrall can then be written as $\int_0^1 h(x)\,dx$.

The only thing one must not forget is to define $h$ such that it does not return the absolute error but only the value of the integral. As written above we do this by adding "[1]" at the end of the quadgk command. (By the way, here you can see that it is really handy to define the function $x/y$ "anonymously" right where we need it.)

In [7]:
h(x)=quadgk(y->x/y,2,3)[1]

quadgk(h,0,1)

(0.2027325540540822, 0.0)